In [1]:
from bs4 import BeautifulSoup
url = 'https://www.tennisexplorer.com/indian-wells/2025/atp-men/'
soup = BeautifulSoup(url, 'html.parser')
print(soup.prettify())

ModuleNotFoundError: No module named 'bs4'